In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [14:54<00:00, 16.38it/s] 


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
239730,Santa Baby (with Henri René & His Orchestra),125,2019-12-07,"Eartha Kitt, Henri Rene & His Orchestra",https://open.spotify.com/track/1foCxQtxBweJtZm...,United States,top200,NEW_ENTRY,309065.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Eartha+Kit...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:1foCxQtxBweJtZmdxhEHVO,0.700,0.139,8.000,-12.346,1.000,0.061,0.902,0.000,0.185,0.490,84.485,audio_features,1foCxQtxBweJtZmdxhEHVO,https://api.spotify.com/v1/tracks/1foCxQtxBweJ...,https://api.spotify.com/v1/audio-analysis/1foC...,206067.000,4.000,22,152,NaN,other
224317,There's No Way (feat. Julia Michaels),92,2018-10-22,Lauv,https://open.spotify.com/track/2hnxrRNzF74mdDz...,United States,top200,MOVE_UP,325199.000,"24 Jan 2022, 00:33","12 Jun 2017, 17:48","Ari Staprans Leff (born August 8, 1994), known...","Ari Staprans Leff (born August 8, 1994), known...",pop,indie pop,indie,american,jazz,colbert,Julia Michaels,NaN,NaN,NaN,spotify:track:3F7bhRhNpY4YKrwa4kGeUz,0.755,0.625,2.000,-7.114,1.000,0.090,0.490,0.000,0.157,0.374,147.022,audio_features,3F7bhRhNpY4YKrwa4kGeUz,https://api.spotify.com/v1/tracks/3F7bhRhNpY4Y...,https://api.spotify.com/v1/audio-analysis/3F7b...,174861.000,4.000,11,69,pop,pop
46237,Red Roses (feat. Landon Cube),43,2018-03-30,Lil Skies,https://open.spotify.com/track/3ZhTT6yjZwpPph5...,United States,top200,MOVE_DOWN,506855.000,"08 Jul 2022, 08:30","18 Dec 2017, 05:08",Lil Skies is an American rapper from Waynesbor...,Lil Skies is an American rapper from Waynesbor...,Hip-Hop,rap,trap,cloud rap,Lil Skies,Lil Skies,Landon Cube,NaN,NaN,NaN,spotify:track:3ZhTT6yjZwpPph5MIJ53XY,0.691,0.683,0.000,-7.335,1.000,0.095,0.547,0.000,0.221,0.456,140.002,audio_features,3ZhTT6yjZwpPph5MIJ53XY,https://api.spotify.com/v1/tracks/3ZhTT6yjZwpP...,https://api.spotify.com/v1/audio-analysis/3ZhT...,262544.000,4.000,36,359,hip-hop,hip-hop
288854,Stuck In A Dream (feat. Gunna),39,2019-10-07,Lil Mosey,https://open.spotify.com/track/5hDAT527wXWNBnJ...,United States,top200,MOVE_UP,481083.000,"01 Sep 2022, 09:18","23 Aug 2018, 05:49","Lathan Moses Echols (born January 25, 2002), k...","Lathan Moses Echols (born January 25, 2002), k...",Hip-Hop,rap,cloud rap,american,lil mosey,2010s,2019,gunna,lil mosey,2010s rap,spotify:track:7iHHxY2NLlJRAmlAmC4ahQ,0.748,0.648,10.000,-6.445,0.000,0.082,0.484,0.000,0.126,0.726,139.952,audio_features,7iHHxY2NLlJRAmlAmC4ahQ,https://api.spotify.com/v1/tracks/7iHHxY2NLlJR...,https://api.spotify.com/v1/audio-analysis/7iHH...,123560.000,4.000,12,74,hip-hop,hip-hop
134547,SLOW DANCING IN THE DARK,136,2021-05-30,Joji,https://open.spotify.com/track/0rKtyWc8bvkriBt...,United States,top200,MOVE_UP,220836.000,"18 Feb 2020, 06:08","04 Jul 2015, 13:40","George Miller, known by his stage name Joji, i...","George Miller, known by his stage name Joji, i...",Lo-Fi,rnb,Hip-Hop,japanese,alternative rnb,soul,rnb,alternative rnb,joji,Hip-Hop,spotify:track:0rKtyWc8bvkriBthvHKY8d,0.515,0.479,3.000,-7.458,1.000,0.026,0.544,0.006,0.191,0.284,88.964,audio_features,0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274.000,4.000,11,1139,lo-fi,lo-fi


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.389
female       0.136
group        0.136
non-binary   0.022
Name: gender, dtype: float64

In [30]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender
152168,Better Now,195,2020-08-18,Post Malone,https://open.spotify.com/track/7dt6x5M1jzdTEt8...,United States,top200,MOVE_DOWN,211454.000,"07 Jan 2022, 23:09","07 Jun 2015, 09:50","Austin Richard Post (born July 4, 1995), known...","Austin Richard Post (born July 4, 1995), known...",Hip-Hop,rap,trap,cloud rap,pop,Hip-Hop,trap,rap,2010s,cloud rap,spotify:track:7dt6x5M1jzdTEt8oCbisTK,0.680,0.578,10.000,-5.804,1.000,0.040,0.331,0.000,0.135,0.341,145.038,audio_features,7dt6x5M1jzdTEt8oCbisTK,https://api.spotify.com/v1/tracks/7dt6x5M1jzdT...,https://api.spotify.com/v1/audio-analysis/7dt6...,231267.000,4.000,1,883,hip-hop,hip-hop,male
215676,Lucky You (feat. Joyner Lucas),114,2018-12-18,Eminem,https://open.spotify.com/track/60SdxE8apGAxMiR...,United States,top200,MOVE_UP,331119.000,"28 May 2019, 23:56","11 Feb 2006, 00:14","Marshall Bruce Mathers III (born October 17, 1...","Marshall Bruce Mathers III (born October 17, 1...",rap,Hip-Hop,Eminem,hip hop,pop,Hip-Hop,rap,joyner lucas,hip hop,2018,spotify:track:60SdxE8apGAxMiRrpbmLY0,0.876,0.786,10.000,-4.884,0.000,0.306,0.063,0.000,0.127,0.575,153.068,audio_features,60SdxE8apGAxMiRrpbmLY0,https://api.spotify.com/v1/tracks/60SdxE8apGAx...,https://api.spotify.com/v1/audio-analysis/60Sd...,244679.000,4.000,1,217,rap,rap,male
416093,Vibe,1,2020-02-23,Cookiee Kawaii,https://open.spotify.com/track/7btQ8AOjr6zXoGH...,United States,viral50,SAME_POSITION,NaN,NaN,"06 Oct 2021, 18:25",Cookiee Kawaii became known outside New Jersey...,Cookiee Kawaii became known outside New Jersey...,jersey club,Hip-Hop,electronic,rap,House,NaN,NaN,NaN,NaN,NaN,spotify:track:4ZAQiu61otvHVveuTsPAUr,0.778,0.576,10.000,-8.698,1.000,0.301,0.063,0.004,0.116,0.155,80.027,audio_features,4ZAQiu61otvHVveuTsPAUr,https://api.spotify.com/v1/tracks/4ZAQiu61otvH...,https://api.spotify.com/v1/audio-analysis/4ZAQ...,84000.000,4.000,1,39,jersey club,other,female
444125,Silence,21,2017-06-01,Grace Carter,https://open.spotify.com/track/6cZzGeLcDeDFuPV...,United States,viral50,MOVE_UP,NaN,NaN,"27 Oct 2018, 16:00",Grace Carter was born in 1997 in London and ra...,Grace Carter was born in 1997 in London and ra...,rnb,pop,british,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:3AweMMF3G598hMsjto9Vtg,0.515,0.122,10.000,-11.192,1.000,0.044,0.806,0.000,0.112,0.164,167.462,audio_features,3AweMMF3G598hMsjto9Vtg,https://api.spotify.com/v1/tracks/3AweMMF3G598...,https://api.spotify.com/v1/audio-analysis/3Awe...,221848.000,4.000,21,6,rnb,rnb,female
17315,No Role Modelz,167,2020-08-04,J. Cole,https://open.spotify.com/track/62vpWI1CHwFy7tM...,United States,top200,MOVE_UP,228289.000,"17 Jan 2020, 09:06","26 Aug 2008, 00:37","Jermaine Lamarr Cole (born January 28, 1985), ...","Jermaine Lamarr Cole (born January 28, 1985), ...",Hip-Hop,rap,underground hip-hop,Jazz Hop,hip hop,Hip-Hop,hip hop,rap,2014,j cole,spotify:track:68Dni7IE4VyPkTOH9mRWHr,0.690,0.521,10.000,-8.492,0.000,0.339,0.324,0.000,0.053,0.494,100.028,audio_features,68Dni7IE4VyPkTOH9mRWHr,https://api.spotify.com/v1/tracks/68Dni7IE4VyP...,https://api.spotify.com/v1/audio-analysis/68Dn...,292799.000,4.000,35,1474,hip-hop,hip-hop,male


Age from artist

-- taking the data from the summary

In [ ]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d*, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

In [ ]:
df.sample()

# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df